- 구조적 API는 비정형 로그 파일부터 반정형 csv, 정형적인 파퀫까지 다양한 데이를 처리할 수 있음
- 다음과 같은 세 가지 분산 컬렉션 API가 존재함
    - 1) Dataset
    - 2) DataFrame
    - 3) SQL table & View
- 배치 및 스트리밍 처리에서 구조적 API를 사용할 수 있으며, 구조적 API를 통해 배치 작업을 스트리밍 작업으로 변환할 수 있음(반대도 OK)
- 구조적 API는 데이터 흐름을 정의하는 기본 추상화 개념
- 이번 장에서 반드시 이해해야하는 세 가지 개념
    - 1) Typed/Untyped API의 개념 및 차이
    - 2) 핵심 용어
    - 3) 스파크가 구조적 API의 데이터 흐름을 해석하고 클러스터에서 실행하는 방식

- Spark : 트랜스포메이션의 처리 과정을 정의하는 프로그래밍 모델
- 사용자가 정의한 다수의 T는 지향성 비순환 그래프(DGA)로 표현되는 명령을 만들어냄
- 액션은 하나의 job을 클러스터에서 실행하기 위해 스테이지와 태스크로 나누고 DAG처리 프로세스를 실행함
- T와 A로 다루는 논리적 구조가 DataFrame & DataSet임
- 새로운 DF나 DES를 만드려면 트랜스포메이션을 호출해야함
    - 그리고 연산을 시작하거나 사용 언어에 맞는 데이터 타입으로 변환하려면 액션을 호출해야함
   

# 4.1.DataFrame & Dataset
- 둘 다 잘 정의된 로우와 컬럼을 가지는 분산 테이블 형태의 컬렉션
    - 각 컬럼은 다른 컬럼과 동일한 수의 로우를 가져야함
    - 또한 각 컬렉션의 모든 로우는 같은 데이터타입 정보를 가지고 있어야함
- 둘 다 결과를 생성하기 위한 지연연산의 실행 계획이며, 불변성을 가짐
- DF에 A를 호출하면 스파크는 T를 실행하고, 결과를 반환함
    - 즉 이 과정은 사용자가 원하는 결과를 얻기 위해 로우와 컬럼을 처리하는 방법에 대한 계획을 나타냄
    - *기본적으로 Table, View는 DF와 동일함*
    - *대신 Table, View는 DF코드 대신 SQL을 활용함*

# 4.2.Schema
- DF의 컬럼명과 데이터타입을 정의함
- 데이터 소스에서 얻거나(Schema on Read), 직접 정의할 수 있음
- 스키마는 여러 데이터 타입으로 구성되므로, 어떤 데이터 타입이 어느 위치에 있는지 정의할 수 있어야함

# 4.3.Introduction To Structural Datatype Of Spark
- 실행 계획의 수립과 처리에 사용하는 자체 데이터타입정보를 가지고 있는 카탈리스트 엔진을 이용함
    - 이는 다양한 실행 최적화 기능을 제공함
- 또한 위 엔진을 통해 자체 데이터타입을 제공하는 여러 언어 API와 매핑할 수 있음
    - 따라서 파이썬이나 R로 스파크의 구조적 API를 이용하더라도, 대다수 연산은 Spark Datatype을 이용하게 됨

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 35.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=45f43ee170198c84ff90d408836c717d483246dfd4c52aeab0828f7bb550f618
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
df = spark.range(500).toDF("number")

In [ ]:
df.select(df["number"] + 10)

DataFrame[(number + 10): bigint]

- 이렇게 pyspark로 실행하더라도 결국 scala기반으로 진행됨
- 이렇게 덧셈연산이 수행될 수 있는 이유는 스파크가 지원하는 언어를 이용해 작성된 표현식을 카탈리스트 엔진에서 스파크의 데이터타입으로 변환하여 명령을 처리하기 때문

### 4.3.1.DataFrame v.s. Dataset
- 본질적으로 구조적 API에는 "비타입형" DataFrame과 "타입형"인 Dataset이 있음
    - DF는 스키마에 명시된 데이터타입과 실제 데이터 타입의 일치여부를 런타임이 되어서야 확인할 수 있음
    - 하지만 DataSet은 컴파일 단계에서 바로 확인함
        - 다만, DataSet은 scala와 Java에서만 지원함
- 이 책의 예제 대부분은 DF를 사용함
- Spark DF는 Row 타입으로 구성된 Dataset임
    - Row타입은 스파크가 사용하는 **연산최적화 인메모리포맷**의 내부적인 표현방식임
    - 이를 통해 가비지 컬렉션과 ㅈ객체 초기화 부하가 있는 JVM데이터 타입을 사용하는 대신, 자체 데이터 포맷을 이용 -> 따라서 **효율성 UP!!**
- 파이썬과 R에서는 Dataset을 이용할 수는 없지만, 최적화된 포맷인 DF로 처리할 수 있음


### 4.3.2.Column
- 스파크의 컬럼은 테이블의 컬럼이라고 생각하면 됨

### 4.3.3.Row
- 데이터 레코드임
- SQL, RDD, DataSource에서 얻거나 만들 수 있음

In [ ]:
spark.range(2).collect()

[Row(id=0), Row(id=1)]

### 4.3.4.DataType Of Spark
- 여러가지 내부 데이터 타입을 지원함
- 특정 데이터타입의 컬럼을 초기화하고 정의하는 방법
    - 스파크와 파이썬의 데이터타입 매칭 여부는 p.116d에서 확인할 수 있음

In [ ]:
from pyspark.sql.types import *
b = ByteType()

![image.png](attachment:image.png)
![image-2.png](attachment:image-2.png)

# 4.4. Procedure Of Structural API
- 스파크 코드가 실제로 어떻게 처리되는지 확인
- 사용자 코드에서 실행 코드로 변환되는 과정은 아래와 같음
    - 1) DF, DS, SQL을 통해 코드 작성
        - console이나 spark-submit셀 스크립트로 실행
    - 2) 정상적인 코드라면 스파크가 **논리적 실행계획**으로 변환함
        - 카탈리스트 옵티마이저가 해당 코드를 넘겨받음
    - 3) 스파크는 **논리적 계획**을 **물리적 실행계획**으로 변환하며 그 과정에서 최적화 진행
        - 카탈리스트 옵티마이저가 실행계획을 생성
    - 4) 스파크는 클러스터에서 **물리적 실행계획**을 실행함(RDDcjfl)
        - 스파크가 코드를 실행하고 결과를 반환
- ![image.png](attachment:image.png)

### 4.4.1. Logical Plan
- 첫 실행 단계로, 사용자 코드를 논리적 실행계획으로 변환함
- ![image.png](attachment:image.png)
    - 위 그림에서는 Optimized Logical Plan까지가 Logical Plan 단계
- 논리적 실행계획단계에서는 추상적인 T만 표현함
    - 즉, 드라이버나 익스큐터의 정보를 고려하지 않음
- 사용자의 다양한 표현식을 최적화된 버전으로 변환함
    - 이 과정에서 **검증 전 논리적 실행 계획(Unresolved Logical Plan)**으로 변환됨
    - 오로지 코드의 유효성과 테이블/컬럼의 존재 여부만 판단하는 과정으로 *실행계획을 검증하지는 않은 상태*
- 그리고 **스파크 분석기**가 컬럼과 테이블을 검증하기 위해 **카탈로그**, 모든 테이블의 저장소, DFwjdqhfmf ghkfdydgkrp ehla
    - 만약 해당 카탈로그에 필요 테이블이나 컬럼이 존재하지 않는 다면, 검증 전 논리적 실행계획 자체가 만들어지지 않음
- 검증 결과는 카탈리스트 옵티마이저로 전달
    - **카탈리스트 옵티마이저** : *조건절 푸시다운*이나 *선택절 구문*을 이용해 논리적 실행 계획을 최적화하는 규칙의 모음


### 4.4.2.Physical Plan
- 스파크 실행계획이라고도 불림
- 논리적 실행계획을 클러스터 환경에서 실행하는 방법을 정의함
- 다양한 물리적 실행 전략을 생성하고 코스트 모델을 이용하여 비교한 후 최적의 모델 선택
- ![image.png](attachment:image.png)
- 물리적 실행계획은 일련의 RDD와 T로 변환됨
    - 그 후 DF, DS, SQL로 정의된 쿼리를 RDD T로 컴파일함

### 4.4.3.Execute
- 최적의 물리적 실행계획을 선정한 다음, 저수준 프로그래밍 인터페이스인 RDD를 대상으로 모든 코드를 실행
    - 이 때 런타임 전체 태스크나 스테이지를 제거할 수 있는 자바 바이트 코드를 생성하여 추가적인 최적화를 수행하고 결과를 반환함